In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('golf.csv')
df

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Outlook      14 non-null     object
 1   Temperature  14 non-null     object
 2   Humidity     14 non-null     object
 3   Windy        14 non-null     bool  
 4   Play         14 non-null     object
dtypes: bool(1), object(4)
memory usage: 590.0+ bytes


In [4]:
df.head()

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [5]:
df.shape

(14, 5)

In [6]:
for label in df.columns:
    df[label] = LabelEncoder().fit(df[label]).transform(df[label])

In [7]:
df.head()

,Outlook,Temperature,Humidity,Windy,Play
0,2,1,0,0,0
1,2,1,0,1,0
2,0,1,0,0,1
3,1,2,0,0,1
4,1,0,1,0,1


In [8]:
df.isna().any()

Outlook        False
Temperature    False
Humidity       False
Windy          False
Play           False
dtype: bool

In [9]:
X = df.drop(['Play'], axis = 1)
y = df['Play']

In [10]:
sets = [train_test_split(X, y, test_size = i / 10, random_state=0) for i in range(1, 6)]

In [11]:
acc_dict = {}

models = [i for i in range(1, 6)]

acc_dict["Test Split"] = []
for i in models:
    acc_dict[f"Test Split"].append(i / 10)
    acc_dict[f"KNN {i}"] = []
acc_dict[f"Ensemble"] = []

for i in enumerate(sets):
    X_train, X_test, y_train, y_test = i[1]
    
    models = []
    
    for j in range(1, 6):
        knn = Pipeline([('m', KNeighborsClassifier(n_neighbors=j, metric='euclidean'))])
        models.append((f'knn{j}', knn))
    
    for j in models:
        y_pred = j[-1].fit(X_train, y_train).predict(X_test)
        acc_dict[f"KNN {(i[0] + 1)}"].append(metrics.accuracy_score(y_test, y_pred))
    
    ensemble = VotingClassifier(estimators = models, voting = 'soft', flatten_transform=True)
    model = ensemble.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    
    acc_dict["Ensemble"].append(metrics.accuracy_score(y_test, y_pred))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

In [13]:
print(acc_dict)
finaldf = pd.DataFrame(acc_dict)
finaldf.to_csv('final_golf.csv')

{'Test Split': [0.1, 0.2, 0.3, 0.4, 0.5], 'KNN 1': [0.5, 0.0, 0.5, 0.5, 0.5], 'KNN 2': [0.0, 0.0, 0.6666666666666666, 0.3333333333333333, 0.6666666666666666], 'KNN 3': [0.2, 0.2, 0.6, 0.4, 0.4], 'KNN 4': [0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333], 'KNN 5': [0.42857142857142855, 0.42857142857142855, 0.42857142857142855, 0.42857142857142855, 0.42857142857142855], 'Ensemble': [0.5, 0.3333333333333333, 0.4, 0.3333333333333333, 0.42857142857142855]}
